In [1]:
import requests
import gzip
import shutil
import os


url = "https://aclanthology.org/anthology+abstracts.bib.gz"
r = requests.get(url)
open('anthology+abstracts.bib.gz', 'wb').write(r.content)

with gzip.open('anthology+abstracts.bib.gz', 'rb') as f_in:
    with open('anthology+abstracts.bib', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
os.remove("anthology+abstracts.bib.gz")

In [ ]:
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove

filepath = 'anthology+abstracts.bib'

fd, abspath = mkstemp()
with fdopen(fd,'w') as repl:
    with open(filepath,'r') as bibtex_file:
        for line in bibtex_file:
            if "month = " in line:
                old_cont = line[12:line.find(',')]
                month = line[12:15]
                new_cont = '"{}"'.format(month)
                #line_quot = 'month = "{}"'.format(line[12:15])
                repl.write(line.replace(old_cont, new_cont))
            else:
                repl.write(line)
            
copymode(filepath, abspath)
remove(filepath)
move(abspath, filepath)

In [2]:
import bibtexparser

with open('anthology+abstracts.bib') as bibtex_file:   
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)


In [27]:
import pandas as pd

keywords = ['fair', 'fairness', 'race', 'gender', 'bias', 'biases']
df = pd.DataFrame()
#df = pd.DataFrame(columns=['title', 'author', 'booktitle', 'month', 'year', 'publisher', 'url', 'doi'])

for paper in bib_database.entries:
    if 'abstract' in paper.keys():
        if any(keyword in paper['abstract'] for keyword in keywords):
            # only select paper from 2016 on
            if 'year' in paper.keys() and int(paper['year']) >= 2016:
            
                for cat in paper.keys():
                    temp_row[cat] = paper[cat]

                df = df.append(temp_row, ignore_index=True)
        
df.to_csv('bias_paper.csv')